In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # 如果你使用Chrome浏览器
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import csv

In [2]:
url1 = 'https://shanghai.anjuke.com/sale'
url2 = '/?pi=sogou-cpc-sh-pinppai2&kwid=%7Bkeywordid%7D&qz_gdt=nyaf4zokdqac6mrlu2iq'
districts = ['pudong', 'minhang', 'songjiang', 'baoshan', 'jiading', 'xuhui', 'qingpu', 'jingan', 'putuo', 'yangpu', 'fengxian', 'huangpu', 'hongkou', 'changning', 'jinshan', 'chongming']
pages = ['','/p2','/p3','/p4','/p5','/p6','p7','p8','p9','p10']
info_text = ['配套成熟','环境优美','繁华地段','安全性高','交通便利','品质小区']
map_floor = {'低':1,'中':2,'高':3}

In [3]:
# 设置WebDriver的路径,不改变
webdriver_path = r'C:\Users\HONOR\Desktop\chromedriver-win64\chromedriver.exe'
def get_data(district,writer,writer_district):
    #初始化Chrome浏览器
    service = Service(webdriver_path)
    driver = webdriver.Chrome(service=service)

    #打开网页
    for page in pages:
        url = url1 +'/' +district +page +url2
        driver.get(url = url)
        
##############################################窗口滚动#######################################################

        scroll_pause_time = 0.5  # 设置滚动间隔时间
        screen_height = driver.execute_script("return document.body.scrollHeight;")# 获取屏幕高度
        # 执行滚动操作
        i = 1
        prev_height = driver.execute_script("return document.body.scrollHeight")
        # 执行滚动操作
        while True:
            driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
            i += 1
            time.sleep(scroll_pause_time)
            # 获取滚动后的页面高度
            new_height = driver.execute_script("return document.body.scrollHeight")  
            # 检查页面高度是否变化
            if new_height == prev_height:
                break  # 如果页面高度未变化，认为已滚动到底部
            prev_height = new_height  # 更新滚动前的页面高度  

#############################################寻找你需要的目标########################################################
        wait = WebDriverWait(driver, 2)  # 设置等待时间为10秒，根据实际情况调整
        house_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.property')))

############################################对目标进行提取############################################################

        for house in house_list:
            sign = False
            info = house.find_elements(By.CLASS_NAME,'property-content-info')
            prices = house.find_element(By.CLASS_NAME,'property-price')
            info0 = info[0].find_elements(By.TAG_NAME, 'p')

            if len(info0)<4:
                continue
            #检测目标目录是否存在
            if len(info)>2:
                info1 = info[2].find_elements(By.TAG_NAME, 'span')
                sign = True
            #检测目标目录


            #提取数据，数据处理
            size = info0[1].text
            floor = info0[3].text
            if floor[0] not in map_floor:
                continue
            year = info0[4].text
            price = prices.find_elements(By.TAG_NAME,'p')[1].text
            print(f"{district},{size},{floor},{year},")
            if sign ==True:
                tag_list = []
                for tag in info1:
                    if tag.text in info_text:
                        print(tag.text,",")
                        tag_list.append(tag.text)
            print(price,"\n")
            
            
            #将数据存储至csv文件中
            data = {
                    'District': district,
                    'Size': re.sub('㎡','',size),
                    'Floor': map_floor[floor[0]],
                    'Year': re.sub('年建造','',year),
                    'Price': re.sub('元/㎡','',price)
                    }
            if sign ==True:
                for tag in info_text:
                    data[tag] = tag in tag_list
            else:
                for tag in info_text:
                    data[tag] = False

            writer.writerow(data)
            writer_district.writerow(data)
            


           
            

    # 关闭浏览器
    driver.quit()




In [4]:
if __name__ == '__main__':

    #创建总csv文件，并创造标题

    with open('house_data.csv', 'a', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['District', 'Size', 'Floor', 'Year'] + info_text  +['Price']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if csv_file.tell() == 0:
            writer.writeheader()

        #对每个地区的房价分别提取，并用不同csv文件存储
        for district in districts:
            with open(district+'.csv','a',newline='', encoding='utf-8') as district_csv:
                writer_district = csv.DictWriter(district_csv, fieldnames=fieldnames)
                if district_csv.tell() == 0:
                    writer_district.writeheader()
                get_data(district,writer,writer_district)


pudong,89.2㎡,低层(共18层),2011年建造,
品质小区 ,
环境优美 ,
54933元/㎡ 

pudong,106.74㎡,中层(共8层),2010年建造,
63332元/㎡ 

pudong,37.88㎡,低层(共6层),1985年建造,
84478元/㎡ 

pudong,84.5㎡,高层(共18层),2006年建造,
繁华地段 ,
安全性高 ,
58580元/㎡ 

pudong,62.44㎡,高层(共6层),1999年建造,
66464元/㎡ 

pudong,84.84㎡,低层(共18层),2016年建造,
57521元/㎡ 

pudong,95.8㎡,中层(共11层),2006年建造,
品质小区 ,
环境优美 ,
112213元/㎡ 

pudong,48.28㎡,高层(共6层),1996年建造,
繁华地段 ,
配套成熟 ,
57788元/㎡ 

pudong,69.19㎡,中层(共6层),1998年建造,
67929元/㎡ 

pudong,127.08㎡,中层(共12层),2007年建造,
配套成熟 ,
交通便利 ,
78691元/㎡ 

pudong,74.31㎡,低层(共6层),1999年建造,
繁华地段 ,
交通便利 ,
59952元/㎡ 

pudong,35㎡,中层(共6层),1997年建造,
繁华地段 ,
配套成熟 ,
59715元/㎡ 

pudong,92.01㎡,中层(共11层),2011年建造,
繁华地段 ,
交通便利 ,
53582元/㎡ 

pudong,142.26㎡,中层(共14层),2007年建造,
繁华地段 ,
配套成熟 ,
79433元/㎡ 

pudong,40㎡,低层(共6层),1999年建造,
繁华地段 ,
配套成熟 ,
54000元/㎡ 

pudong,58.74㎡,高层(共6层),2000年建造,
配套成熟 ,
55329元/㎡ 

pudong,138.92㎡,中层(共14层),2007年建造,
品质小区 ,
繁华地段 ,
110136元/㎡ 

pudong,135.88㎡,低层(共18层),2005年建造,
品质小区 ,
配套成熟 ,
安全性高 ,
86032元/㎡ 

pudong,98.42㎡,高层(共6层),1998年建造,
55883元/㎡ 

pudong,84.72㎡